# Minimum code for Extended Data Figure 4 qPCR panel

In [ ]:
options(warn = -1)
suppressPackageStartupMessages({
  library(tidyverse)
  library(rasilabRtemplates)
})

In [ ]:
if (!file.exists('../figures')){
    dir.create('../figures')
}
    

## Read in annotations

In [ ]:
annotations <- read_csv("../annotations/sampleannotations.csv") %>% 
  mutate(Well = as.character(Well),
         Plate = as.character(Plate)) %>%
  print()

## Read in Ct data; join with annotations

In [ ]:
cq_data <- read_csv("../../../../data/qpcr/nmd_reporter_validation/cq_values.csv") %>%
   select(Cq, Well) %>%
   rename(CT = Cq) %>%
   inner_join(annotations) %>%
   select(CT, Amplicon, Treatment, Reporter) %>%
   mutate(Treatment = fct_relevel(Treatment, "DMSO")) %>%
   print()

## Normalize β-globin expression to mCherry

In [ ]:
norm_ct_inverted <- cq_data  %>%
  filter(!is.na(CT)) %>%
  group_by(Amplicon, Treatment, Reporter) %>%
  summarize(std_error = sd(CT),
            n = n(),
            CT = mean(CT)) %>%
  ungroup() %>%
  group_by(Treatment, Reporter) %>%
  mutate(norm_ct = CT - CT[Amplicon == "mCherry"],
         norm_error = sqrt(std_error^2 + std_error[Amplicon == "mCherry"]^2)) %>%
  filter(Amplicon == "globin") %>%
  mutate(norm_ct = -5 - norm_ct) %>%
  print()

## Plot normalized β-globin reporter expression

In [ ]:
options(repr.plot.width=3, repr.plot.height=2)

norm_ct_inverted %>% 
  ggplot(aes(x = Treatment, y = norm_ct, ymax = norm_ct + norm_error, ymin = norm_ct - norm_error, color = Reporter)) +
  geom_point() +
  geom_errorbar(width = 0.2) +
  ylab("log2 mRNA level\n(-ΔΔCT)") +
  theme(axis.text.x = element_text(),
        axis.line = element_line(color = "grey")) +
  scale_x_discrete(guide = guide_axis(angle = 30)) +
  scale_color_manual(values = cbPalette) 

ggsave("../figures/mcherry_normalized_ct_values_inverted.pdf", height = 2, width = 3) 